<a href="https://colab.research.google.com/github/saipragna25/deep-learning-pipeline-custom-components/blob/main/DL_pipeline_custom_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt


Load the MNIST dataset:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

custom loss function

In [ ]:
# Custom loss function
class CustomLoss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        return tf.reduce_mean(tf.square(y_true - y_pred))

# Model with custom loss function
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=CustomLoss(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 62s 36ms/step - loss: 27.2875 - accuracy: 0.0963 - val_loss: 27.4584 - val_accuracy: 0.0990
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 27.2875 - accuracy: 0.1020 - val_loss: 27.4584 - val_accuracy: 0.1048
Epoch 3/5
1688/1688 [==============================] - 58s 34ms/step - loss: 27.2875 - accuracy: 0.1014 - val_loss: 27.4584 - val_accuracy: 0.0955
Epoch 4/5
1688/1688 [==============================] - 65s 39ms/step - loss: 27.2875 - accuracy: 0.1002 - val_loss: 27.4584 - val_accuracy: 0.0953
Epoch 5/5
1688/1688 [==============================] - 55s 33ms/step - loss: 27.2874 - accuracy: 0.1007 - val_loss: 27.4584 - val_accuracy: 0.0953


Custom Regularizer

In [ ]:
# Custom regularizer
class CustomRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, l1=0.0, l2=0.0):
        self.l1 = l1
        self.l2 = l2

    def __call__(self, x):
        loss = 0.0
        if self.l1:
            loss += tf.reduce_sum(self.l1 * tf.abs(x))
        if self.l2:
            loss += tf.reduce_sum(self.l2 * tf.square(x))
        return loss

# Model with custom regularizer
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=CustomRegularizer(l1=1e-5, l2=1e-4)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=CustomRegularizer(l1=1e-5, l2=1e-4)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=CustomRegularizer(l1=1e-5, l2=1e-4)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 62s 36ms/step - loss: 0.2435 - accuracy: 0.9483 - val_loss: 0.1237 - val_accuracy: 0.9847
Epoch 2/5
1688/1688 [==============================] - 59s 35ms/step - loss: 0.1284 - accuracy: 0.9815 - val_loss: 0.1014 - val_accuracy: 0.9907
Epoch 3/5
1688/1688 [==============================] - 58s 35ms/step - loss: 0.1117 - accuracy: 0.9840 - val_loss: 0.0901 - val_accuracy: 0.9910
Epoch 4/5
1688/1688 [==============================] - 59s 35ms/step - loss: 0.0978 - accuracy: 0.9871 - val_loss: 0.0891 - val_accuracy: 0.9897
Epoch 5/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0917 - accuracy: 0.9870 - val_loss: 0.0881 - val_accuracy: 0.9887


Custom activation function

In [ ]:
# Custom activation function
@tf.function
def custom_activation(x):
    return tf.maximum(0.0, x) * tf.sin(x)

# Model with custom activation function
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1)),
    layers.Activation(custom_activation),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3)),
    layers.Activation(custom_activation),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128),
    layers.Activation(custom_activation),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 76s 45ms/step - loss: 0.1826 - accuracy: 0.9444 - val_loss: 0.0430 - val_accuracy: 0.9893
Epoch 2/5
1688/1688 [==============================] - 73s 43ms/step - loss: 0.0494 - accuracy: 0.9855 - val_loss: 0.0392 - val_accuracy: 0.9887
Epoch 3/5
1688/1688 [==============================] - 75s 44ms/step - loss: 0.0338 - accuracy: 0.9896 - val_loss: 0.0333 - val_accuracy: 0.9902
Epoch 4/5
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0235 - accuracy: 0.9925 - val_loss: 0.0314 - val_accuracy: 0.9905
Epoch 5/5
1688/1688 [==============================] - 74s 44ms/step - loss: 0.0183 - accuracy: 0.9942 - val_loss: 0.0357 - val_accuracy: 0.9902


  Custom Initializer

In [ ]:
class CustomInitializer(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        return tf.random.normal(shape, mean=0., stddev=0.05)

# Model with custom initializer
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_initializer=CustomInitializer()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_initializer=CustomInitializer()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_initializer=CustomInitializer()),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 61s 36ms/step - loss: 0.1721 - accuracy: 0.9474 - val_loss: 0.0474 - val_accuracy: 0.9873
Epoch 2/5
1688/1688 [==============================] - 66s 39ms/step - loss: 0.0565 - accuracy: 0.9824 - val_loss: 0.0403 - val_accuracy: 0.9893
Epoch 3/5
1688/1688 [==============================] - 65s 39ms/step - loss: 0.0398 - accuracy: 0.9879 - val_loss: 0.0344 - val_accuracy: 0.9893
Epoch 4/5
1688/1688 [==============================] - 60s 35ms/step - loss: 0.0309 - accuracy: 0.9901 - val_loss: 0.0337 - val_accuracy: 0.9917
Epoch 5/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0247 - accuracy: 0.9920 - val_loss: 0.0267 - val_accuracy: 0.9932


Custom kernel constraint

In [ ]:
class CustomConstraint(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return tf.clip_by_value(w, -1.0, 1.0)

# Model with custom kernel constraint
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_constraint=CustomConstraint()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_constraint=CustomConstraint()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_constraint=CustomConstraint()),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 57s 33ms/step - loss: 0.1579 - accuracy: 0.9508 - val_loss: 0.0536 - val_accuracy: 0.9848
Epoch 2/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0541 - accuracy: 0.9835 - val_loss: 0.0435 - val_accuracy: 0.9878
Epoch 3/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0380 - accuracy: 0.9881 - val_loss: 0.0368 - val_accuracy: 0.9902
Epoch 4/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0283 - accuracy: 0.9907 - val_loss: 0.0488 - val_accuracy: 0.9865
Epoch 5/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.0235 - accuracy: 0.9924 - val_loss: 0.0333 - val_accuracy: 0.9913


Custom Metrics

In [ ]:
class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name="custom_accuracy", **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name="correct", initializer="zeros")
        self.total = self.add_weight(name="total", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        self.correct.assign_add(tf.reduce_sum(tf.cast(y_true == y_pred, tf.float32)))
        self.total.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.correct / self.total

# Model with custom metrics
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[CustomAccuracy()])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 59s 35ms/step - loss: 0.1588 - custom_accuracy: 0.0984 - val_loss: 0.0439 - val_custom_accuracy: 0.0967
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0524 - custom_accuracy: 0.0990 - val_loss: 0.0350 - val_custom_accuracy: 0.0975
Epoch 3/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0376 - custom_accuracy: 0.0990 - val_loss: 0.0340 - val_custom_accuracy: 0.0972
Epoch 4/5
1688/1688 [==============================] - 55s 32ms/step - loss: 0.0287 - custom_accuracy: 0.0989 - val_loss: 0.0340 - val_custom_accuracy: 0.0968
Epoch 5/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0228 - custom_accuracy: 0.0988 - val_loss: 0.0311 - val_custom_accuracy: 0.0975


Custom Seed

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Set the custom seed
custom_seed = 42
np.random.seed(custom_seed)
tf.random.set_seed(custom_seed)

# Model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.1523 - accuracy: 0.9524 - val_loss: 0.0511 - val_accuracy: 0.9857
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0539 - accuracy: 0.9834 - val_loss: 0.0428 - val_accuracy: 0.9870
Epoch 3/5
1688/1688 [==============================] - 55s 33ms/step - loss: 0.0375 - accuracy: 0.9882 - val_loss: 0.0294 - val_accuracy: 0.9915
Epoch 4/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0276 - accuracy: 0.9912 - val_loss: 0.0339 - val_accuracy: 0.9907
Epoch 5/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.0320 - val_accuracy: 0.9907


Custom Call Back

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.95:
            print("\nReached 95% accuracy, stopping training.")
            self.model.stop_training = True

# Model with custom callback
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1, callbacks=[CustomCallback()])


Custom Layer

In [ ]:
class CustomLayer(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Model with custom layer
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    CustomLayer(128),
    layers.Activation('relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Model

In [ ]:
class CustomModel(models.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.max_pool1 = layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.max_pool2 = layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu')
        self.dropout = layers.Dropout(0.2)
        self.dense2 = layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

# Model with custom model
model = CustomModel()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Training Loop

In [ ]:
# Model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Custom training loop
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

for epoch in range(5):
    for step, (images, labels) in enumerate(train_dataset):
        loss = train_step(images, labels)
        if step % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss {loss.numpy()}")


Custom Dropout

In [ ]:
class CustomDropout(layers.Layer):
    def __init__(self, rate=0.2, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

# Model with custom dropout
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    CustomDropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Learning Rate Scheduler

In [ ]:
def custom_learning_rate_schedule(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.1
    return lr

# Model with custom learning rate scheduler
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

callback = tf.keras.callbacks.LearningRateScheduler(custom_learning_rate_schedule)

model.fit(x_train, y_train, epochs=5, validation_split=0.1, callbacks=[callback])


Epoch 1/5
1688/1688 [==============================] - 62s 36ms/step - loss: 0.1579 - accuracy: 0.9515 - val_loss: 0.0508 - val_accuracy: 0.9835 - lr: 0.0010
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0535 - accuracy: 0.9839 - val_loss: 0.0405 - val_accuracy: 0.9888 - lr: 0.0010
Epoch 3/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0369 - accuracy: 0.9884 - val_loss: 0.0334 - val_accuracy: 0.9900 - lr: 0.0010
Epoch 4/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0270 - accuracy: 0.9916 - val_loss: 0.0376 - val_accuracy: 0.9898 - lr: 0.0010
Epoch 5/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0224 - accuracy: 0.9930 - val_loss: 0.0330 - val_accuracy: 0.9913 - lr: 0.0010
